# FBSDE

Ji, Shaolin, Shige Peng, Ying Peng, and Xichuan Zhang. “Three Algorithms for Solving High-Dimensional Fully-Coupled FBSDEs through Deep Learning.” ArXiv:1907.05327 [Cs, Math], February 2, 2020. http://arxiv.org/abs/1907.05327.

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Reshape, concatenate, Layer
from keras import Model, initializers
from keras.callbacks import ModelCheckpoint
from keras.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  3


In [14]:
# numerical parameters
n_paths = 2 ** 18
n_timesteps = 4
n_dimensions = 4
n_diffusion_factors = 2
n_jump_factors = 2
T = 1.
dt = T / n_timesteps
batch_size = 128
epochs = 1000

In [33]:
# model parameters
eta = 1.
lp = 1.
lm = 1.
k = 1.
sigma = 1.
zeta = 1.
phi = 1.
psi = 1.
epsilon = 5e-3

# Initial value layer

In [34]:
class InitialValue(Layer):
    
    def __init__(self, y0, **kwargs):
        super().__init__(**kwargs)
        self.y0 = y0
    
    def call(self, inputs):
        return self.y0

# Coefficients

In [35]:
def b(t, x, y, z, r):
    
    ad = y[2] / y[3] + x[0]
    dp = tf.maximum(0., 1./k + ad)
    dm = tf.maximum(0., 1./k - ad)
    
    return [
        x[1],
        -eta * x[0],
        lp * tf.exp(-k * dp) - lm * tf.exp(-k * dm),
        lp * (x[0] + dp) * tf.exp(-k * dp) - lm * (x[0] - dp) * tf.exp(-k * dm),
    ]

def s(t, x, y, z, r):
    return [[sigma, 0], [0, zeta], [0, 0], [0, 0]]

# - dH_dx
def f(t, x, y, z, r):
    
    ad = y[2] / y[3] + x[0]
    dp = tf.maximum(0., 1./k + ad)
    dm = tf.maximum(0., 1./k - ad)

    return [
        -(y[3] * lp * tf.exp(-k * dp) - y[3] * lm * tf.exp(-k * dm)),
        -(y[0] - eta * y[1]),
        -(-2. * phi * x[2]),
        -(0.)
    ]

def v(t, x, y, z, r):
    return [[0, 0], [epsilon, -epsilon], [0, 0], [0, 0]]

# dg_dx
def g(x):
    return [x[2], 0., x[0] - 2 * psi * x[2], 1.]

In [36]:
def dX(i, x, y, z, r, dW, dN):
    
    t = i * dt
    
    def drift(arg):
        x, y, z, r = arg
        return tf.math.multiply(b(t, x, y, z, r), dt)
    a0 = tf.vectorized_map(drift, (x, y, z, r))
        
    def noise(arg):
        x, y, z, r, dW = arg
        return tf.tensordot(s(t, x, y, z ,r), dW[i], [[1], [0]])
    a1 = tf.vectorized_map(noise, (x, y, z, r, dW))

    def jump(arg):
        x, y, z, r, dN = arg
        return tf.tensordot(v(t, x, y, z ,r), dN[i], [[1], [0]])
    a2 = tf.vectorized_map(jump, (x, y, z, r, dN))
    
    return a0 + a1 + a2

def dY(i, x, y, z, r, dW, dN):
    
    t = i * dt

    def drift(arg):
        x, y, z, r = arg
        return tf.math.multiply(f(t, x, y, z, r), dt)
    a0 = tf.vectorized_map(drift, (x, y, z, r))

    def noise(arg):
        x, y, z, r, dW = arg
        return tf.tensordot(z, dW[i], [[1], [0]])
    a1 = tf.vectorized_map(noise, (x, y, z, r, dW))
    
    def jump(arg):
        x, y, z, r, dN = arg
        return tf.tensordot(r, dN[i], [[1], [0]])
    a2 = tf.vectorized_map(jump, (x, y, z, r, dN))
    
    return a0 + a1 + a2

# Model

In [37]:
paths = []

inputs_dW = Input(shape=(n_timesteps, n_diffusion_factors))
inputs_dN = Input(shape=(n_timesteps, n_jump_factors))

x0 = tf.Variable([[0., 0., 0., 0.]], trainable=False)
y0 = tf.Variable([[5., 5., 5., 5.]], trainable=True)

x = InitialValue(x0, name='x_0')(inputs_dW)
y = InitialValue(y0, name='y_0')(inputs_dW)

z = concatenate([x, y])
z = Dense(10, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='z1_0')(z)
z = Dense(n_dimensions * n_diffusion_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='z2_0')(z)
z = Reshape((n_dimensions, n_diffusion_factors), name='zr_0')(z)

r = concatenate([x, y])
r = Dense(10, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='r1_0')(r)
r = Dense(n_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='r2_0')(r)
r = Reshape((n_dimensions, n_jump_factors), name='rr_0')(r)

paths += [[x, y, z, r]]

@tf.function
def hx(i, r):
    return r[0] + dX(i, *r)

@tf.function
def hy(i, r):
    return r[1] + dY(i, *r)

for i in range(n_timesteps):
    
    x, y = (
        Lambda(lambda r: hx(i, r), name=f'x_{i+1}')([x, y, z, r, inputs_dW, inputs_dN]),
        Lambda(lambda r: hy(i, r), name=f'y_{i+1}')([x, y, z, r, inputs_dW, inputs_dN]),
    )
    
    # we don't train z for the last time step; keep for consistency
    z = concatenate([x, y])
    z = Dense(10, activation='relu', name=f'z1_{i+1}')(z)
    z = Dense(n_dimensions * n_diffusion_factors, activation='relu', name=f'z2_{i+1}')(z)
    z = Reshape((n_dimensions, n_diffusion_factors), name=f'zr_{i+1}')(z)
    
    # we don't train r for the last time step; keep for consistency
    r = concatenate([x, y])
    r = Dense(10, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name=f'r1_{i+1}')(r)
    r = Dense(n_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name=f'r2_{i+1}')(r)
    r = Reshape((n_dimensions, n_jump_factors), name=f'rr_{i+1}')(r)

    paths += [[x, y, z, r]]
    
outputs_loss = Lambda(lambda r: r[1] - tf.transpose(tf.vectorized_map(g, r[0])))([x, y])
outputs_paths = tf.stack(
    [tf.stack([p[0] for p in paths[1:]], axis=1), tf.stack([p[1] for p in paths[1:]], axis=1)] + 
    [tf.stack([p[2][:, :, i] for p in paths[1:]], axis=1) for i in range(n_diffusion_factors)] +
    [tf.stack([p[3][:, :, i] for p in paths[1:]], axis=1) for i in range(n_jump_factors)], axis=2)

model_loss = Model([inputs_dW, inputs_dN], outputs_loss)
model_loss.compile(loss='mse', optimizer='adam')

# (n_sample, n_timestep, x/y/z_k, n_dimension)
# skips the first time step
model_paths = Model([inputs_dW, inputs_dN], outputs_paths)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [38]:
model_loss.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 4, 2)]       0                                            
__________________________________________________________________________________________________
x_0 (InitialValue)              (1, 4)               4           input_5[0][0]                    
__________________________________________________________________________________________________
y_0 (InitialValue)              (1, 4)               4           input_5[0][0]                    
__________________________________________________________________________________________________
concatenate_20 (Concatenate)    (1, 8)               0           x_0[0][0]                        
                                                                 y_0[0][0]                  

# Training

In [39]:
dW = tf.sqrt(dt) * tf.random.normal((n_paths, n_timesteps, n_diffusion_factors))
dN = tf.random.poisson((n_paths, n_timesteps), [dt * lp, dt * lm])
target = tf.zeros((n_paths, n_dimensions))

In [40]:
# check for exploding gradients before training

with tf.GradientTape() as tape:
    loss = model_loss([dW, dN])

# bias of the last dense layer
variables = model_loss.variables[-1]
tape.gradient(loss, variables)

The following Variables were used a Lambda layer's call (x_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 4) dtype=float32, numpy=array([[        5,         5,         5,         5]], dtype=float32)>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (y_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 4) dtype=float32, numpy=array([[        5,         5,         5,         5]], dtype=float32)>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


<tf.Tensor: shape=(8,), dtype=float32, numpy=array([        0, 1.435e+04, 6.563e+04,        29,         0,       838,  5.68e+04, 3.544e+04], dtype=float32)>

In [ ]:
callback = ModelCheckpoint('_models/weights{epoch:04d}.h5', save_weights_only=True, overwrite=True)
model_loss.save_weights('_models/weights0000.h5')
history = model_loss.fit([dW, dN], target, batch_size=batch_size, epochs=1000, callbacks=[callback])


Epoch 1/1000
2048/2048 [==============================] - 12s 4ms/step - loss: 22.0765
Epoch 2/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 31.7059
Epoch 3/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 19.1041
Epoch 4/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 18.4369
Epoch 5/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 18.1734
Epoch 6/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 17.9959
Epoch 7/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 18.2402
Epoch 8/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 17.9759
Epoch 9/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 17.8844
Epoch 10/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 17.5983
Epoch 11/1000
2048/2048 [==============================] - 8s 4ms/step - loss: 17.1111
Epoch 12/1000
2048/2048 [==========================

In [24]:
model_loss.variables[1]

<tf.Variable 'Variable:0' shape=(1, 4) dtype=float32, numpy=array([[2.1957033, 1.416712 , 3.749401 , 0.2265092]], dtype=float32)>

# Display paths and loss

In [183]:
# load bad model
model_loss.load_weights('_models/weights0001.h5')

In [25]:
loss = model_loss([dW, dN]).numpy()
loss

array([[-0.05399251,  0.19388664, -0.06744671, -0.7734908 ],
       [ 0.18906283,  0.19388664,  0.04259288, -0.7734908 ],
       [ 3.4222124 ,  0.19388664,  0.3228495 ,  0.05090344],
       ...,
       [ 0.18894172,  0.04759699, -0.10360461, -0.12479764],
       [-0.12686372,  0.04088575, -0.09768331,  0.5292703 ],
       [ 0.02140617,  0.19388664,  0.40589857, -0.7734908 ]],
      dtype=float32)

In [26]:
paths = model_paths([dW, dN]).numpy()

In [32]:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%9.4g" % x))
tf.transpose(paths[50, :, :, :], (1, 2, 0)).numpy()

array([[[ -0.07307,   -0.1896,   -0.3449,   -0.5318],
        [  -0.1737,   -0.3291,   -0.4554,   -0.5428],
        [    -0.25,      -0.5,     -0.75,        -1],
        [    4.141,     8.281,      12.1,     14.81]],

       [[    2.252,     2.309,     2.366,   -0.9582],
        [    1.222,    0.9644,    0.6282,    0.1127],
        [    3.749,     3.455,     2.459,     1.659],
        [   0.2265,    0.2265,    0.2265,    0.2265]],

       [[        0,         0,     14.56,         0],
        [        0,         0,     1.111,      1.16],
        [    2.324,     2.425,    0.9417,         0],
        [        0,         0,         0,     4.925]],

       [[        0,         0,     13.73,         0],
        [        0,         0,         0,         0],
        [        0,      3.37,     2.115,         0],
        [        0,         0,         0,     4.103]],

       [[        0,         0,         0,         0],
        [        0,         0,         0,         0],
        [        0, 